In [1]:
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import HTML
import pandas as pd
import random
import copy

In [2]:

class FourLinkArm:
    
    def __init__(self):
        # Set parameters for the 3-link planar arm
        self.theta1 = np.deg2rad(0)
        self.theta2 = np.deg2rad(90)
        self.theta3 = np.deg2rad(90)
        self.theta4 = np.deg2rad(90)
        self.orig_l1 = np.random.randint(4,9)
        self.orig_l2 = np.random.randint(4,9)
        self.orig_l3 = np.random.randint(4,9)
        self.orig_l4 = np.random.randint(4,9)
        self.l1=copy.copy(self.orig_l1)
        self.l2=copy.copy(self.orig_l2)
        self.l3=copy.copy(self.orig_l3)
        self.l4=copy.copy(self.orig_l4)
        max_len_shift=int(np.min([self.l1,self.l2,self.l3,self.l4])*(2/3))
        self.arr1=np.linspace(-max_len_shift,max_len_shift,np.random.randint(10,50))
        self.arr1=np.append(self.arr1,np.flip(self.arr1))
        self.permute=np.random.permutation([1,1,-1,-1])

    def forwardKinematics(self, theta1, theta2, theta3,theta4):
        # Define the homogeneous transformation matrices for the 4-link planar arm
        self.theta1 = theta1
        self.theta2 = theta2
        self.theta3 = theta3
        self.theta4 = theta4
        self.t01 = np.matrix([[np.cos(self.theta1), -np.sin(self.theta1), 0], 
                              [np.sin(self.theta1), np.cos(self.theta1), 0],
                              [0, 0, 1]])
        self.t12 = np.matrix([[np.cos(self.theta2), -np.sin(self.theta2), self.l1],
                              [np.sin(self.theta2), np.cos(self.theta2), 0],
                              [0, 0, 1]])
        self.t23 = np.matrix([[np.cos(self.theta3), -np.sin(self.theta3), self.l2],
                              [np.sin(self.theta3), np.cos(self.theta3), 0],
                              [0, 0, 1]])        
        self.t34 = np.matrix([[np.cos(self.theta4), -np.sin(self.theta4), self.l3],
                              [np.sin(self.theta4), np.cos(self.theta4), 0],
                              [0, 0, 1]])
        self.t4end = np.matrix([[np.cos(self.theta4), -np.sin(self.theta4), self.l4],
                                [np.sin(self.theta4), np.cos(self.theta4), 0],
                                [0, 0, 1]])

        self.t0end = self.t01*self.t12*self.t23*self.t34*self.t4end
        return self.t0end
    
    def findJointPos(self): 
        # Find the x,y position of each joint and end effector so it can be plotted
        # Find the transformation matrices for joint 2 and joint 3 and joint 4
        self.t02 = self.t01*self.t12
        self.t03 = self.t01*self.t12*self.t23
        self.t04 = self.t01*self.t12*self.t23*self.t34
        # Find the x, y coordinates for joints 2 and 3 and 4. Put them in a list j2 = [x,y]
        j2 = [ self.t02[0, 2], self.t02[1, 2] ]
        j3 = [ self.t03[0, 2], self.t03[1, 2] ]
        j4 = [ self.t04[0, 2], self.t04[1, 2] ]
        endeff = [self.t0end[0,2],self.t0end[1,2]]
        
        self.jnt2pos, self.jnt3pos, self.jnt4pos, self.endEffPos = j2,j3,j4,endeff
        return j2,j3,j4,endeff
    
    def geomJacobian(self, jnt2pos=None, jnt3pos=None, jnt4pos=None, endEffPos=None):
        if jnt2pos is None:
            jnt2pos=self.jnt2pos 
        if jnt3pos is None:
            jnt3pos=self.jnt3pos
        if jnt4pos is None:
            jnt4pos=self.jnt4pos
        if endEffPos is None:
            endEffPos=self.endEffPos
        ai = np.array([0,0,1])
        col0 = np.array(endEffPos + [0])
        col1 = np.array(endEffPos + [0]) - np.array(jnt2pos + [0])
        col2 = np.array(endEffPos + [0]) - np.array(jnt3pos + [0])
        col3 = np.array(endEffPos + [0]) - np.array(jnt4pos + [0])
        J = np.array([np.cross(ai,col0), np.cross(ai,col1), np.cross(ai,col2), np.cross(ai,col3)]).T 
        return J
    
    def random_link_len_change(self,i):
        self.l1=self.orig_l1+self.permute[0]*self.arr1[i%len(self.arr1)]
        self.l2=self.orig_l2+self.permute[1]*self.arr1[i%len(self.arr1)]
        self.l3=self.orig_l3+self.permute[2]*self.arr1[i%len(self.arr1)]
        self.l4=self.orig_l4+self.permute[3]*self.arr1[i%len(self.arr1)]

In [3]:
def plotArm(jnt2pos, jnt3pos, jnt4pos, endEffectPos, target=None, step=None):
    # set up figure
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, autoscale_on=False, xlim=(-25, 25), ylim=(-25, 25))
    ax.grid()
    
    if step is not None:
        step = np.array(step)
        plt.scatter(step[:, 0],step[:, 1], color='lightblue', marker='o')
        plt.plot(step[:, 0],step[:, 1], color='lightblue')
    if target is not None:
        target = np.array(target)
        plt.scatter(target[:, 0],target[:, 1], color='red', marker='o')
        
    line, = ax.plot([], [], 'o-', lw=4, mew=5)
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

    line.set_data([], [])
    time_text.set_text('')
    x = np.array([0, jnt2pos[0], jnt3pos[0], jnt4pos[0], endEffectPos[0]])
    y = np.array([0, jnt2pos[1], jnt3pos[1], jnt4pos[1], endEffectPos[1]])
    line.set_data((x,y))
    
    plt.show()
    
# gif animation plotter
def plotArm2(jnt2pos, jnt3pos, jnt4pos, endEffectPos,x2,y2, target=None, step=None, fig=None, camera=None):
    # set up figure
    ax = fig.add_subplot(111, autoscale_on=False,
                         xlim=(-25, 25), ylim=(-25, 25))
    ax.grid()
    line, = ax.plot([], [], 'o-', lw=4, mew=5, color='lightblue')
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

    line.set_data([], [])
    time_text.set_text('')
    x = np.array([0, jnt2pos[0], jnt3pos[0], jnt4pos[0], endEffectPos[0]])
    y = np.array([0, jnt2pos[1], jnt3pos[1], jnt4pos[1], endEffectPos[1]])
    line.set_data((x,y))
    
    plt.plot([jnt3pos[0],x2[0]],[jnt3pos[1],y2[0]],'o-', lw=2, mew=5, color='black')
    plt.plot([jnt3pos[0],x2[1]],[jnt3pos[1],y2[1]],'o-', lw=2, mew=5, color='black')


    if step is not None:
        step = np.array(step)
        plt.scatter(step[:, 0],step[:, 1], color='lightblue', marker='o')
        plt.plot(step[:, 0],step[:, 1], color='lightblue')
    if target is not None:
        target = np.array(target)
        plt.scatter(target[:, 0],target[:, 1], color='red', marker='o')

    camera.snap()

In this part, let's plan some simple polynomial trajectories in the work space. 

First, setup a simple environment with the arm manipulator and four intermediate interpolation points.

In [4]:
def create_simulation(animation_index,samples):
    
    # Setup our new manipulator
    arm = FourLinkArm()
    
    # define the initial configuration of the manipulator (you are welcomed to try other values)
    initTheta = [np.deg2rad(0),np.deg2rad(45),np.deg2rad(-45),np.deg2rad(45)]

    # Initialise the arm with the initial values
    _ = arm.forwardKinematics(*initTheta)

    # Get the initial end-effector position 
    init_joint2pos, init_joint3pos, init_joint4pos, EF_init_pos = arm.findJointPos()

    # Define the target postions of the end-effector
    # EF_target_pos = np.array([[EF_init_pos[0], 6.,2.,-4.,-8.],
    #                           [EF_init_pos[1], -6.,8.,-5.,6.]]).T
    ##Choose points randomly, choose x equally spaced and y randomly
    L=arm.l1+arm.l2+arm.l3+arm.l4
    print(L)
    x_points=np.array([2/3*L, 1/3*L, -1/3*L, -2/3*L])

    y_points=[]
    R=3*L/4
    for i in range(4):
        y_points.append(random.uniform(-1*np.sqrt(R**2-x_points[i]**2),
                                          np.sqrt(R**2-x_points[i]**2)))
    y_points=np.array(y_points)
    
    EF_target_pos = np.array([[EF_init_pos[0], x_points[0],x_points[1],x_points[2],x_points[3]],
                              [EF_init_pos[1], y_points[0],y_points[1],y_points[2],y_points[3]]]).T

    # Plot the pose of the arm
    #plotArm(*arm.findJointPos(), target=EF_target_pos)
    print(f"The interpolation points are stored in the variable \"EF_target_pos\": \n{EF_target_pos}")
    
    pol = np.polyfit(EF_target_pos[:, 0], EF_target_pos[:, 1], EF_target_pos.shape[0]-1)  # polynomial coefficients
    
    # Plotting the planned trajectory
    #epsilon = 100
    epsilon = samples
    
    xx = np.linspace(EF_target_pos[0,0],EF_target_pos[-1,0], epsilon)
    yy = np.polyval(pol,xx)   
    curve = np.hstack([xx[:, None], yy[:, None]]) 
    
    interpolated_points=curve

#     plotArm(*arm.findJointPos(), target=EF_target_pos, step=interpolated_points)
    print(f"The intermediate points are stored in the variable \"interpolated_points\"")
    
    # set the current E-F position for the IK loop
    joint2pos, joint3pos, joint4pos, endEffectorPos = init_joint2pos, init_joint3pos, init_joint4pos, EF_init_pos
    # set the initial robot configuration
    newTheta = initTheta

    # Set up the animation generator
    fig = plt.figure(figsize=(10,10))
    camera = Camera(fig)
    
    temp_eeff_angle=np.append(np.linspace(0,45,int(interpolated_points.shape[0]/8)),
                          np.linspace(45,0,int(interpolated_points.shape[0]/8)))
    
    wrist_len_orig=np.random.randint(2,arm.l3+1)
    
    keypoints_data=pd.DataFrame(columns=range((5+2)*2),dtype=float)
    arr2=np.linspace(-int(wrist_len_orig/2),int(wrist_len_orig/2),np.random.randint(10,50))
    
    arr2=np.append(arr2,np.flip(arr2))

    
    for i in range(interpolated_points.shape[0]):
        J = arm.geomJacobian()

        newgoal = interpolated_points[i, :]
        deltaStep = newgoal - endEffectorPos
        subtarget = np.array([deltaStep[0], deltaStep[1], 0]) 

        radTheta = np.linalg.pinv(J) @ subtarget
        newTheta = newTheta + radTheta

        # ----------- Do forward kinematics to plot the arm ---------------
        arm.random_link_len_change(i)
        wrist_len=wrist_len_orig+arr2[i%len(arr2)]

        _ = arm.forwardKinematics(newTheta[0],newTheta[1],newTheta[2],newTheta[3])
        joint2pos, joint3pos, joint4pos, endEffectorPos = arm.findJointPos()

        x2=[]
        y2=[]

        x2=x2+[arm.l1*np.cos(newTheta[0]) \
               +arm.l2*np.cos(newTheta[0]+newTheta[1]) \
               +wrist_len*np.cos(newTheta[0]+newTheta[1]+newTheta[2]+np.deg2rad(temp_eeff_angle[i%len(temp_eeff_angle)]))]
    
        y2=y2+[arm.l1*np.sin(newTheta[0]) \
               +arm.l2*np.sin(newTheta[0]+newTheta[1]) \
               +wrist_len*np.sin(newTheta[0]+newTheta[1]+newTheta[2]+np.deg2rad(temp_eeff_angle[i%len(temp_eeff_angle)]))]
        
        x2=x2+[arm.l1*np.cos(newTheta[0]) \
               +arm.l2*np.cos(newTheta[0]+newTheta[1]) \
               +wrist_len*np.cos(newTheta[0]+newTheta[1]+newTheta[2]-np.deg2rad(temp_eeff_angle[i%len(temp_eeff_angle)]))]
    
        y2=y2+[arm.l1*np.sin(newTheta[0]) \
               +arm.l2*np.sin(newTheta[0]+newTheta[1]) \
               +wrist_len*np.sin(newTheta[0]+newTheta[1]+newTheta[2]-np.deg2rad(temp_eeff_angle[i%len(temp_eeff_angle)]))]

        temp_keypoints=np.array([0,0,joint2pos[0],joint2pos[1],joint3pos[0],joint3pos[1],joint4pos[0],joint4pos[1],
                                 endEffectorPos[0],endEffectorPos[1]])
        
        temp_keypoints=np.append(temp_keypoints,np.array([x2[0],y2[0],x2[1],y2[1]]))
        
        keypoints_data = keypoints_data.append(pd.DataFrame(temp_keypoints[None,:],columns=keypoints_data.columns),
                                               ignore_index = True)
        
        plotArm2(joint2pos, joint3pos, joint4pos, endEffectorPos,x2,y2, target=EF_target_pos, fig=fig, camera=camera)


    plt.close()
    animation = camera.animate()
    animation.save('./animated_motion_dynamic_len/animated_motion'+str(animation_index)+'.gif', writer = 'imagemagick')
    
    # create the adjacency matrix
    adj_mat=np.zeros((5+2,5+2))
    adj_mat[0][1]=1
    adj_mat[1][2]=1
    adj_mat[2][3]=1
    adj_mat[3][4]=1
    adj_mat[2][5]=1
    adj_mat[2][6]=1
    
    pd.DataFrame(adj_mat).astype(int).to_csv('./animated_motion_dynamic_len/adj_mat'+str(animation_index)+'.csv')
    keypoints_data.to_csv('./animated_motion_dynamic_len/ts_data'+str(animation_index)+'.csv')    
    

In [5]:
import os
os.environ['MAGICK_THREAD_LIMIT'] = '2'

In [6]:
# create_simulation(1026,50)

In [7]:
for i in range(750,875):
    create_simulation(i,50)

25
The interpolation points are stored in the variable "EF_target_pos": 
[[ 21.48528137   8.48528137]
 [ 16.66666667   6.71991786]
 [  8.33333333   4.88967146]
 [ -8.33333333   7.20619298]
 [-16.66666667   7.79231615]]
The intermediate points are stored in the variable "interpolated_points"


<ipython-input-3-9b304879dac9>:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(111, autoscale_on=False,


23
The interpolation points are stored in the variable "EF_target_pos": 
[[ 1.94852814e+01  8.48528137e+00]
 [ 1.53333333e+01  7.33765161e-03]
 [ 7.66666667e+00 -1.01787317e+01]
 [-7.66666667e+00  1.34706844e+01]
 [-1.53333333e+01  7.14732818e-01]]
The intermediate points are stored in the variable "interpolated_points"


<ipython-input-3-9b304879dac9>:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(111, autoscale_on=False,


19
The interpolation points are stored in the variable "EF_target_pos": 
[[ 16.65685425   5.65685425]
 [ 12.66666667   1.51044936]
 [  6.33333333  12.21876618]
 [ -6.33333333   0.1464217 ]
 [-12.66666667   0.27362291]]
The intermediate points are stored in the variable "interpolated_points"


...

<ipython-input-3-9b304879dac9>:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(111, autoscale_on=False,


21
The interpolation points are stored in the variable "EF_target_pos": 
[[ 17.48528137   8.48528137]
 [ 14.          -2.45706362]
 [  7.           4.17484379]
 [ -7.          -5.99437032]
 [-14.          -1.82372896]]
The intermediate points are stored in the variable "interpolated_points"


<ipython-input-3-9b304879dac9>:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(111, autoscale_on=False,


28
The interpolation points are stored in the variable "EF_target_pos": 
[[ 23.89949494   9.89949494]
 [ 18.66666667   1.28298238]
 [  9.33333333  -4.70323431]
 [ -9.33333333  -8.91131885]
 [-18.66666667  -5.65035159]]
The intermediate points are stored in the variable "interpolated_points"


<ipython-input-3-9b304879dac9>:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(111, autoscale_on=False,
